# Two-Tier Urban Deliveries with Robots with no time windows

## Gurobi Model

In [1]:
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter  # for some testing [optional]
import ipywidgets as widgets  # for interactive plots [optional]
from ipywidgets import interact, interact_manual  # for interactive plots [optional]
from tqdm.notebook import tqdm  # for progress bars [optional]

# program must be adjusted in some places if optional packages are not available

#### Some custom functions for later use

In [2]:
def grid (a, n=None, spacing=None):
    """
    distribute n points evenly in a 2-dimensional square of side length a
    or create a square grid with suqare cells of length c
    """
    if spacing is None:
        assert n
        spacing = a // int(np.sqrt(n))
        base = range(spacing//2, a, spacing)
    if n is None:
        assert spacing
        base = range(0, a+1, spacing)
    return[(x,y) for x in base for y in base]

def travel_time(first: tuple, second:tuple, speed):
    """
    compute the travel time between first and second
    first, second: given as (x, y) tuples 
    speed: given in km/h
    """
    dist = abs(first[0] - second[0]) + abs(first[1] - second[1])  # Manhattan distance in m (!) because input is in meters
    return (dist/1000)/speed  # travel time in hours



In [23]:
grid(2000, spacing=100)

[(0, 0),
 (0, 100),
 (0, 200),
 (0, 300),
 (0, 400),
 (0, 500),
 (0, 600),
 (0, 700),
 (0, 800),
 (0, 900),
 (0, 1000),
 (0, 1100),
 (0, 1200),
 (0, 1300),
 (0, 1400),
 (0, 1500),
 (0, 1600),
 (0, 1700),
 (0, 1800),
 (0, 1900),
 (0, 2000),
 (100, 0),
 (100, 100),
 (100, 200),
 (100, 300),
 (100, 400),
 (100, 500),
 (100, 600),
 (100, 700),
 (100, 800),
 (100, 900),
 (100, 1000),
 (100, 1100),
 (100, 1200),
 (100, 1300),
 (100, 1400),
 (100, 1500),
 (100, 1600),
 (100, 1700),
 (100, 1800),
 (100, 1900),
 (100, 2000),
 (200, 0),
 (200, 100),
 (200, 200),
 (200, 300),
 (200, 400),
 (200, 500),
 (200, 600),
 (200, 700),
 (200, 800),
 (200, 900),
 (200, 1000),
 (200, 1100),
 (200, 1200),
 (200, 1300),
 (200, 1400),
 (200, 1500),
 (200, 1600),
 (200, 1700),
 (200, 1800),
 (200, 1900),
 (200, 2000),
 (300, 0),
 (300, 100),
 (300, 200),
 (300, 300),
 (300, 400),
 (300, 500),
 (300, 600),
 (300, 700),
 (300, 800),
 (300, 900),
 (300, 1000),
 (300, 1100),
 (300, 1200),
 (300, 1300),
 (300, 1400)

#### Model inputs

In [3]:
np.random.seed(0)  # for reproducable results

n = 100  # number of clients
h = 16  # number of potential hubs (should be a square number (4, 9, 16, 25, ...) for an even distribution)
r_max = 5  # number of robots per hub
num_instances = 10

# downtown
area= 2000  # 2km * 2km square
block = 100  # 100m * 100m blocks

M = 6  # maximum allowed driving time per robot (hrs)
tf = 45/60  # robots' full recharge time (hrs)
b = 2  # robots' battery range (hrs)
ts = 4/60  # customer service time (hrs)
v = 3  # robot speed (km/h)

H = [i for i in range(h)]  # hubs
R = [i for i in range(r_max)]  # robots
N = [i for i in range(n)]  # customers
I = [i for i in range(num_instances)]  # instances

hub_coords = grid(a=area, n=h)

grid_x, grid_y = zip(*grid(a=area, spacing=block))
customer_coords = gp.tupledict()
for instance in I:  # originally, smaller instances are subsets of the bigger ones (n=300), here all instances are unique atm
    for customer in N:
        customer_coords[instance, customer] = (np.random.choice(grid_x), np.random.choice(grid_y))


t = gp.tupledict()  # pendulum distances
for hub in tqdm(H, desc='Pendulum distances'):
    for robot in R:
        for instance in I:
            for customer in N:
                t[hub, robot, instance, customer] = 2*travel_time(first=hub_coords[hub],
                                                                  second=customer_coords[instance, customer],
                                                                  speed=v)
            
reachables = [key for (key, value) in t.items() if value <=b]  # customers within battery range of each hub

reachable_customers = gp.tupledict()  # dictionary of customers that are in reach of each (hub, robot, instance) combination
for hub in tqdm(H, desc='Reachable customers'):
    for robot in R:
        for instance in I:
            in_reach = []
            for customer in N:    
                if t.select(hub, robot, instance, customer)[0] <= b:
                    in_reach.append(customer)
                reachable_customers[(hub, robot, instance)] = in_reach

#### Plotting hubs and a single instance

In [4]:
colors = plt.get_cmap('hsv', h)
hubs_x, hubs_y = zip(*hub_coords)

def base_plot(plot_instance:int):
    plt.scatter(hubs_x, hubs_y, cmap=colors, c=H, marker='s', s=75, edgecolors='black')
    if plot_instance is not None:
        customers_x, customers_y = zip(*customer_coords.select(plot_instance, '*'))
        plt.scatter(customers_x, customers_y, c='black')
        for customer in N:
            plt.annotate(s=customer, xy=(customers_x[customer], customers_y[customer]))
    plt.xlim(0, area)
    plt.ylim(0, area)
    # plt.gca().set_xticks(np.arange(0, area+block, block), minor=True)
    # plt.gca().set_yticks(np.arange(0, area+block, block), minor=True)
    # plt.grid(which='minor', dashes=(2, 10))
    # plt.grid(which='major')
    # plt.gcf().set_size_inches(15,8)  
interact(base_plot, plot_instance=[None]+I);

interactive(children=(Dropdown(description='plot_instance', options=(None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9), valu…

### Tier 1: Minimize the number of hubs

#### Model, decision variables and objective function

In [5]:
model1 = gp.Model('min_hubs')

# add hub-robot-instance-customer binary decision variables
x1 = model1.addVars(reachables, vtype=GRB.BINARY, name='x')
# format of x1 vars: (hub, instance, customer, robot)

# add is-hub-open binary decision variables
o1 = model1.addVars(H, vtype=GRB.BINARY, name='o')

model1.setObjective(gp.quicksum(o1), sense=GRB.MINIMIZE)
model1.update()

Using license file C:\Users\Elting\gurobi.lic
Academic license - for non-commercial use only


#### Adding constraints

In [6]:
# (2) All instance-customer locations must be assigned to exactly one hub-robot combination
for instance in I:
    for customer in N:
        model1.addConstr(x1.sum('*', '*', instance, customer) == 1)

In [7]:
# (3) limit on the maximum robot working time. First create a dictionary with the respective coefficients then add linear expression
coeff1 = gp.tupledict({
    (hub, robot, instance, customer): t[hub, robot, instance, customer] * (1 + (tf / b)) + ts
    for hub in H
    for robot in R
    for instance in I
    for customer in N
})

for hub in H:
    for robot in R:
        for instance in I:
            model1.addConstr(x1.prod(coeff1, hub, robot, instance, '*') <= M)

In [8]:
# (4) if a robot serves a customer location (in some instance), the corresponding robot hub is open
for hub in H:
    for instance in I:
        for customer in reachable_customers[hub, robot, instance]:
            model1.addConstr(x1.sum(hub, '*', instance, customer) <= o1[hub])

#### Solve Model 1

In [9]:
model1.setParam('TimeLimit', 60)  # for testing only
model1.optimize()

Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 17658 rows, 79306 columns and 253728 nonzeros
Model fingerprint: 0x62d8e3ae
Variable types: 0 continuous, 79306 integer (79306 binary)
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 16.0000000
Presolve time: 0.51s
Presolved: 17658 rows, 79306 columns, 253728 nonzeros
Variable types: 0 continuous, 79306 integer (79306 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 1.000000e+00, 14776 iterations, 1.35 seconds
Total elapsed time = 14.62s
Total elapsed time = 19.73s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl U

#### Inspect and plot one of the instances with the solution

In [10]:
x1_solution = model1.getAttr('x', x1)
assignment1 = gp.tupledict({
    (key[2], key[3]):(key[0], key[1]) 
    for key, value in x1_solution.items() 
    if value > 0.5})  # (instance, customer): (hub, robot)

In [11]:
o1_solution = model1.getAttr('x', o1)
p = len([key for key, value in o1_solution.items() if value > 0.5])  # min number of open hubs
p

5

In [12]:
def model1_plot(plot_instance):
    colors = plt.get_cmap('hsv', p)
    hub_colors = []
    for hub in H:
        if o1_solution[hub] > 0.5:
            hub_colors.append(colors(hub/h))
        else:
            hub_colors.append('white')

    plt.scatter(hubs_x, hubs_y, c=hub_colors, marker='s', s=75, edgecolors='black')
    
    if plot_instance is not None:
        customer_colors = []
        for customer in N:
            customer_colors.append(colors(assignment1[plot_instance, customer][0]/h))  

        customers_x, customers_y = zip(*customer_coords.select(plot_instance, '*'))  
        plt.scatter(customers_x, customers_y, c=customer_colors, alpha=1, edgecolors='black')

        if n <= 100:
            for customer in N:
                plt.annotate(s=customer, xy=(customers_x[customer], customers_y[customer]))

    plt.xlim(0, area)
    plt.ylim(0, area);
    # plt.gcf().set_size_inches(20,10)
interact(model1_plot, plot_instance=[None]+I);

interactive(children=(Dropdown(description='plot_instance', options=(None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9), valu…

### Tier 2: Minimize operational robot cost

#### Model, decision variables, objective function

In [13]:
model2 = gp.Model('min_cost')

# add decision variables
x2 = model2.addVars(reachables, vtype=GRB.BINARY, name='x')
o2 = model2.addVars(H, vtype=GRB.BINARY, name= 'o')

# set the objective function
coeff2 = t

model2.setObjective(x2.prod(coeff2), sense=GRB.MINIMIZE)
model2.update()
# model2.getObjective()

#### Add constraints

In [14]:
# define the constraints
# (2) All customer locations must be assigned to exactly one hub-robot combination (same as in first model)
for instance in I:
    for customer in N:
        model2.addConstr(x2.sum('*', '*', instance, customer) == 1)
        
# (3) limit the robot max working time (same as in first model)
for hub in H:
    for robot in R:
        for instance in I:
            model2.addConstr(x2.prod(coeff1, hub, robot, instance, '*') <= M)
        
# (4) Hub is open when one customer is served by any of its robots ????? (4) in paper
for hub in H:
    for instance in I:
        for customer in reachable_customers[hub, robot, instance]:
            model2.addConstr(x2.sum(hub, '*', instance, customer) <= o2[hub])

# (8) ensure that there are exactly as many open robot hubs as provided by first model
model2.addConstr(o2.sum() == p)

model2.update()

#### Solve Model 2

In [15]:
model2.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 17659 rows, 79306 columns and 253744 nonzeros
Model fingerprint: 0x0393305a
Variable types: 0 continuous, 79306 integer (79306 binary)
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [7e-02, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Presolve time: 0.49s
Presolved: 17659 rows, 79306 columns, 253744 nonzeros
Variable types: 0 continuous, 79306 integer (79306 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 3.224000e+02, 4818 iterations, 1.20 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  322.40000    0 1324          -  322.40000      -     -    2s
H    0     0                     365.8666667  322.

#### Inspect and plot solution of Model 2

In [16]:
objValue2 = model2.getObjective().getValue()
objValue2

322.8666666666651

In [17]:
x2_solution = model2.getAttr('x', x2)
# customer: (hub, robot) lookup dict
assignment2 = gp.tupledict({
    (key[2], key[3]):(key[0], key[1]) 
    for key, value in x2_solution.items() 
    if value > 0.5})
assignment2
# test whether the constraints are satisfied: every customer is assigned to one robot and one hub
Counter([x2_solution.sum('*', '*', instance, customer).getValue() == 1 for customer in N for instance in I])

Counter({True: 1000})

In [18]:
# test the constraints: maximum working time of robots
Counter([x2.prod(coeff1, hub, instance, '*', robot).getValue() <= M for hub in H for instance in I for robot in R])

Counter({True: 800})

In [19]:
o2_solution = model2.getAttr('x', o2)
o2_solution
# test whether constraints are satisfied: only three hubs are open
o2_solution.sum().getValue() == p

True

In [29]:
workload = dict()
for hub in H:
    for robot in R:
        for instance in I:
            wl = x2_solution.sum(hub, robot, instance, '*').getValue()
            if wl > 0:
                customers = [key[1] for key, value in assignment2.items() if value == (hub, robot) and key[0] == instance]
                workload[hub, robot, instance] = [customers,
                                        wl,
                                        x2_solution.prod(coeff2, hub, instance, '*', robot).getValue(),  # pure travel time over all instances
                                        x2_solution.prod(coeff1, hub, instance, '*', robot).getValue()  # total time incl. recharge etc.
                                       ]
active_robots, wl_customers, wl_num_customers, wl_travel_time, wl_total_time = gp.multidict(workload)
# active_robots, wl_customers, wl_num_customers, wl_travel_time, wl_total_time

In [21]:
def model2_plot(plot_instance):
    # plot hubs
    hub_colors = []
    for hub in H:
        if o2_solution[hub] > 0.5:
            hub_colors.append(colors(hub/h))
        else:
            hub_colors.append('black')
    plt.scatter(hubs_x, hubs_y, c=hub_colors, marker='s', s=75, edgecolors='black')
    
    # plot customers
    customer_colors = []
    for customer in N:
        customer_colors.append(colors(assignment2[plot_instance, customer][0]/h))
    customers_x, customers_y = zip(*customer_coords.select(plot_instance, '*'))
    plt.scatter(customers_x, customers_y, c=customer_colors, alpha=0.5, edgecolors='black')

    for customer in N:
        plt.annotate(s=customer, xy=(customers_x[customer], customers_y[customer]))

    # plot robot tours, different line style per robot
    dashes = [[1, 0], [5,5], [1,1], [3,1,1,1], [3,10,1,10]]  # may have to be extended for r_max > 5
    for hub, robot, instance in active_robots.select('*', '*', plot_instance):
        C = wl_customers[hub, robot, instance]
        for customer in C:
            plt.plot([hubs_x[hub], customers_x[customer]], [hubs_y[hub], customers_y[customer]],
                     c=hub_colors[hub], dashes=dashes[robot], zorder=0
                    )

    plt.gcf().set_size_inches(20,10)
    plt.gca().set_xticks(np.arange(0, area+block, block))
    plt.gca().set_yticks(np.arange(0, area+block, block))
    plt.grid()
    # obviously, this plot does not show the actual tours which are based on manhatten distances
interact(model2_plot, plot_instance=I);

interactive(children=(Dropdown(description='plot_instance', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0), …

In [28]:
workload

{(2, 0, 0): [[15, 24, 28, 43, 45, 54, 56, 65, 72, 76, 82, 85, 93],
  13.0,
  0.4666666666666666,
  0.7083333333333333],
 (2, 0, 1): [[2, 11, 17, 18, 26, 30, 32, 64, 65, 76], 10.0, 0.0, 0.0],
 (2, 0, 2): [[23, 27, 28, 41, 43, 47, 49, 51, 63, 81, 86, 95], 12.0, 0.0, 0.0],
 (2, 0, 3): [[5, 10, 16, 22, 24, 34, 37, 43, 56, 65, 69], 11.0, 0.0, 0.0],
 (2, 0, 4): [[12, 13, 19, 22, 45, 67, 80, 81, 83, 86, 92], 11.0, 0.0, 0.0],
 (2, 0, 5): [[23, 31, 45, 46, 65, 81, 83, 89, 96, 99], 10.0, 0.0, 0.0],
 (2, 0, 6): [[0, 3, 30, 37, 44, 48, 58, 65, 75, 79, 84, 87, 97, 99],
  14.0,
  0.0,
  0.0],
 (2, 0, 7): [[1, 8, 11, 14, 20, 26, 27, 32, 35, 44, 62, 66, 71],
  13.0,
  0.0,
  0.0],
 (2, 0, 8): [[16, 29, 30, 43, 53, 67, 68, 77, 78, 81, 84, 88, 91, 98],
  14.0,
  0.0,
  0.0],
 (2, 0, 9): [[8, 17, 20, 23, 29, 49, 65, 88, 90], 9.0, 0.0, 0.0],
 (2, 1, 0): [[57, 80], 2.0, 0.26666666666666666, 0.4333333333333333],
 (2, 1, 1): [[27, 62, 78, 85, 93, 97, 99], 7.0, 0.0, 0.0],
 (2, 1, 2): [[38, 58], 2.0, 0.0, 0.0]